# CS579: Lecture 13  

**Demographic Inference II**

*[Dr. Aron Culotta](http://cs.iit.edu/~culotta)*  
*[Illinois Institute of Technology](http://iit.edu)*

## Gender Classification

Let's build a classifier to predict whether a Twitter user is male/female.

We'll collect "labeled" training data using Census name list.

**1.) Collect Census names. **

In [1]:
# Fetch male/female names from Census.

import requests

def get_census_names():
    """ Fetch a list of common male/female names from the census.
    For ambiguous names, we select the more frequent gender."""
    males = requests.get('http://www2.census.gov/topics/genealogy/1990surnames/dist.male.first').text.split('\n')
    females = requests.get('http://www2.census.gov/topics/genealogy/1990surnames/dist.female.first').text.split('\n')
    males_pct = dict([(m.split()[0].lower(), float(m.split()[1]))
                  for m in males if m])
    females_pct = dict([(f.split()[0].lower(), float(f.split()[1]))
                    for f in females if f])
    male_names = set([m for m in males_pct if m not in females_pct or
                  males_pct[m] > females_pct[m]])
    female_names = set([f for f in females_pct if f not in males_pct or
                  females_pct[f] > males_pct[f]])    
    return male_names, female_names

male_names, female_names = get_census_names()
print('found %d female and %d male names' % (len(male_names), len(female_names)))
print('male name sample:', list(male_names)[:5])
print('female name sample:', list(female_names)[:5])

found 1146 female and 4014 male names
male name sample: ['lemuel', 'loren', 'luciano', 'ernesto', 'frank']
female name sample: ['delaine', 'maye', 'cristie', 'alyssa', 'felica']


**2.) Sample 5K tweets with names on the Census list. **

In [2]:
# Construct TwitterAPI object.

import configparser
from TwitterAPI import TwitterAPI

def get_twitter(config_file):
    """ Read the config_file and construct an instance of TwitterAPI.
    Args:
      config_file ... A config file in ConfigParser format with Twitter credentials
    Returns:
      An instance of TwitterAPI.
    """
    #config = configparser.ConfigParser()
    #config.read(config_file)
    #twitter = TwitterAPI(
    #               config.get('twitter', 'consumer_key'),
    #               config.get('twitter', 'consumer_secret'),
    #               config.get('twitter', 'access_token'),
    #               config.get('twitter', 'access_token_secret'))
    
    #my info
    consumer_key = 'Q6sCDic9j8mNMhZB2BysN69vr'
    consumer_secret = 'FdaQjrFThC0F3xddTZ1WnwhNmCtw08es0X1EIMCloQmOfxUc47'
    access_token = '3584891536-Lm7t8eHMyf9l0MDXVdXwL4gj2xKZJ6EHzxp85ev'
    access_token_secret = 'cajDlqGO2ExbHh0cRLK4WOyzw1i5E7Q4EIIi4qe99EoG2'
    twitter = TwitterAPI(consumer_key, consumer_secret, access_token, access_token_secret)
    return twitter


twitter = get_twitter('twitter.cfg')

In [3]:
# Sample U.S. tweets with names from Census. 
import sys

def get_first_name(tweet):
    if 'user' in tweet and 'name' in tweet['user']:
        parts = tweet['user']['name'].split()
        if len(parts) > 0:
            return parts[0].lower()

def sample_tweets(twitter, limit, male_names, female_names):
    tweets = []
    while True:
        try:
            # Restrict to U.S.
            for response in twitter.request('statuses/filter',
                        {'locations':'-124.637,24.548,-66.993,48.9974'}):
                if 'user' in response:
                    name = get_first_name(response)
                    if name in male_names or name in female_names:# get the names in our census list
                        tweets.append(response)
                        if len(tweets) % 100 == 0:
                            print('found %d tweets' % len(tweets))
                        if len(tweets) >= limit:
                            return tweets
        except:
            print("Unexpected error:", sys.exc_info()[0])
    return tweets
        
tweets = sample_tweets(twitter, 5000, male_names, female_names)

found 100 tweets
found 200 tweets
found 300 tweets
found 400 tweets
found 500 tweets
found 600 tweets
found 700 tweets
found 800 tweets
found 900 tweets
found 1000 tweets
found 1100 tweets
found 1200 tweets
found 1300 tweets
found 1400 tweets
found 1500 tweets
found 1600 tweets
found 1700 tweets
found 1800 tweets
found 1900 tweets
found 2000 tweets
found 2100 tweets
found 2200 tweets
found 2300 tweets
found 2400 tweets
found 2500 tweets
found 2600 tweets
found 2700 tweets
found 2800 tweets
found 2900 tweets
found 3000 tweets
found 3100 tweets
found 3200 tweets
found 3300 tweets
found 3400 tweets
found 3500 tweets
found 3600 tweets
found 3700 tweets
found 3800 tweets
found 3900 tweets
found 4000 tweets
found 4100 tweets
found 4200 tweets
found 4300 tweets
found 4400 tweets
found 4500 tweets
found 4600 tweets
found 4700 tweets
found 4800 tweets
found 4900 tweets
found 5000 tweets


In [4]:
from collections import Counter

print('sampled %d tweets' % len(tweets))
print('top names:', Counter(get_first_name(t) for t in tweets).most_common(10))

sampled 5000 tweets
top names: [('mike', 59), ('john', 50), ('chris', 46), ('james', 41), ('matt', 37), ('david', 37), ('king', 36), ('katie', 34), ('michael', 34), ('mark', 32)]


In [5]:
# Save these tweets.
import pickle
pickle.dump(tweets, open('tweets.pkl', 'wb'))

**3.) Tokenize tweets. **

In [19]:
test_tweet = tweets[201]
print('test tweet:\n\tscreen_name=%s\n\tname=%s\n\tdescr=%s\n\ttext=%s' %
      (test_tweet['user']['screen_name'],
       test_tweet['user']['name'],
       test_tweet['user']['description'],
       test_tweet['text']))

test tweet:
	screen_name=thompsontroy
	name=troy thompson
	descr=Love that grace, struggling sinner/saint, lover of all peoples, follower of a Lamb, teller of hope, believer of miracles, provider of rest to the ugly birds!
	text=Merry Christmas from precious_._thompson! #christmas #northcoast @ Church on the North Coast https://t.co/HBXTwiE1Wc


In [20]:
import re

def tokenize(string, lowercase, keep_punctuation, prefix,
             collapse_urls, collapse_mentions):
    """ Split a tweet into tokens."""
    if not string:
        return []
    if lowercase:
        string = string.lower()
    tokens = []
    if collapse_urls:
        string = re.sub('http\S+', 'THIS_IS_A_URL', string)
    if collapse_mentions:
        string = re.sub('@\S+', 'THIS_IS_A_MENTION', string)
    if keep_punctuation:
        tokens = string.split()
    else:
        tokens = re.sub('\W+', ' ', string).split()
    if prefix:
        tokens = ['%s%s' % (prefix, t) for t in tokens]
    return tokens

In [21]:
tokenize(test_tweet['user']['description'], lowercase=True,
         keep_punctuation=True, prefix='d=',
         collapse_urls=True, collapse_mentions=True)

['d=love',
 'd=that',
 'd=grace,',
 'd=struggling',
 'd=sinner/saint,',
 'd=lover',
 'd=of',
 'd=all',
 'd=peoples,',
 'd=follower',
 'd=of',
 'd=a',
 'd=lamb,',
 'd=teller',
 'd=of',
 'd=hope,',
 'd=believer',
 'd=of',
 'd=miracles,',
 'd=provider',
 'd=of',
 'd=rest',
 'd=to',
 'd=the',
 'd=ugly',
 'd=birds!']

In [22]:
tokenize(test_tweet['text'], lowercase=True, keep_punctuation=True,
         prefix='t=',
         collapse_urls=True, collapse_mentions=False)

['t=merry',
 't=christmas',
 't=from',
 't=precious_._thompson!',
 't=#christmas',
 't=#northcoast',
 't=@',
 't=church',
 't=on',
 't=the',
 't=north',
 't=coast',
 't=THIS_IS_A_URL']

In [23]:
def tweet2tokens(tweet, use_descr=True, lowercase=True,
                 keep_punctuation=True, descr_prefix='d=',
                 collapse_urls=True, collapse_mentions=True):
    """ Convert a tweet into a list of tokens, from the tweet text and optionally the
    user description. """
    tokens = tokenize(tweet['text'], lowercase, keep_punctuation, None,
                       collapse_urls, collapse_mentions)
    if use_descr:
        tokens.extend(tokenize(tweet['user']['description'], lowercase,
                               keep_punctuation, descr_prefix,
                               collapse_urls, collapse_mentions))
    return tokens

In [24]:
# for enumerating all possible arguments of tweet2tokens
# https://docs.python.org/2/library/itertools.html#itertools.product
from itertools import product

use_descr_opts = [True, False]
lowercase_opts = [True, False]
keep_punctuation_opts = [True, False]
descr_prefix_opts = ['d=', '']
url_opts = [True, False]
mention_opts = [True, False]

argnames = ['use_descr', 'lower', 'punct', 'prefix', 'url', 'mention']
option_iter = product(use_descr_opts, lowercase_opts,
                       keep_punctuation_opts,
                       descr_prefix_opts, url_opts,
                       mention_opts)
for options in option_iter:
    print('  '.join('%s=%s' % (name, opt) for name, opt in zip(argnames, options)))
    print
    print('  '.join(tweet2tokens(test_tweet, *options)), '\n----\n')
    


use_descr=True  lower=True  punct=True  prefix=d=  url=True  mention=True
merry  christmas  from  precious_._thompson!  #christmas  #northcoast  @  church  on  the  north  coast  THIS_IS_A_URL  d=love  d=that  d=grace,  d=struggling  d=sinner/saint,  d=lover  d=of  d=all  d=peoples,  d=follower  d=of  d=a  d=lamb,  d=teller  d=of  d=hope,  d=believer  d=of  d=miracles,  d=provider  d=of  d=rest  d=to  d=the  d=ugly  d=birds! 
----

use_descr=True  lower=True  punct=True  prefix=d=  url=True  mention=False
merry  christmas  from  precious_._thompson!  #christmas  #northcoast  @  church  on  the  north  coast  THIS_IS_A_URL  d=love  d=that  d=grace,  d=struggling  d=sinner/saint,  d=lover  d=of  d=all  d=peoples,  d=follower  d=of  d=a  d=lamb,  d=teller  d=of  d=hope,  d=believer  d=of  d=miracles,  d=provider  d=of  d=rest  d=to  d=the  d=ugly  d=birds! 
----

use_descr=True  lower=True  punct=True  prefix=d=  url=False  mention=True
merry  christmas  from  precious_._thompson!  #chris

In [25]:
# Let's tokenize all tweets.
tokens_list = [tweet2tokens(t, use_descr=True, lowercase=True,
                            keep_punctuation=False, descr_prefix='d=',
                            collapse_urls=True, collapse_mentions=True)
              for t in tweets]

In [26]:
tokens_list[201]

['merry',
 'christmas',
 'from',
 'precious_',
 '_thompson',
 'christmas',
 'northcoast',
 'church',
 'on',
 'the',
 'north',
 'coast',
 'THIS_IS_A_URL',
 'd=love',
 'd=that',
 'd=grace',
 'd=struggling',
 'd=sinner',
 'd=saint',
 'd=lover',
 'd=of',
 'd=all',
 'd=peoples',
 'd=follower',
 'd=of',
 'd=a',
 'd=lamb',
 'd=teller',
 'd=of',
 'd=hope',
 'd=believer',
 'd=of',
 'd=miracles',
 'd=provider',
 'd=of',
 'd=rest',
 'd=to',
 'd=the',
 'd=ugly',
 'd=birds']

In [27]:
# Store these in a sparse matrix.

#1) Create a vocabulary (dict from term->index)

# https://docs.python.org/2/library/collections.html#collections.defaultdict
from collections import defaultdict

def make_vocabulary(tokens_list):
    vocabulary = defaultdict(lambda: len(vocabulary))  # If term not present, assign next int. 
    #just give them a one by one index
    for tokens in tokens_list:
        for token in tokens:
            vocabulary[token]  # looking up a key; defaultdict takes care of assigning it a value.
    print('%d unique terms in vocabulary' % len(vocabulary))
    return vocabulary

In [28]:
vocabulary = make_vocabulary(tokens_list)

19261 unique terms in vocabulary


In [29]:
# term->index
list(vocabulary.items())[:10]

[('d=calling', 2830),
 ('are', 158),
 ('boostmobile', 15769),
 ('d=davis', 17890),
 ('d=cocks', 14823),
 ('d=kentucky', 6595),
 ('d=woody', 6672),
 ('yazoohefeisthegreatestbeerintheknownuniverse', 11140),
 ('d=gmg', 9836),
 ('d=22', 856)]

In [38]:
# How big is vocabulary if we keep punctuation?
tokens_list = [tweet2tokens(t, use_descr=True, lowercase=True,
                            keep_punctuation=True, descr_prefix='d=',
                            collapse_urls=True, collapse_mentions=True)
              for t in tweets]

vocabulary = make_vocabulary(tokens_list)

27237 unique terms in vocabulary


use_descr=True  lower=True  punct=False  prefix=d=  url=True  mention=True

merry  christmas  from  precious_  _thompson  christmas  northcoast  church  on  the  north  coast  THIS_IS_A_URL  d=love  d=that  d=grace  d=struggling  d=sinner  d=saint  d=lover  d=of  d=all  d=peoples  d=follower  d=of  d=a  d=lamb  d=teller  d=of  d=hope  d=believer  d=of  d=miracles  d=provider  d=of  d=rest  d=to  d=the  d=ugly  d=birds 

use_descr=True  lower=True  punct=True  prefix=d=  url=True  mention=True

merry  christmas  from  precious_._thompson!  #christmas  #northcoast  @  church  on  the  north  coast  THIS_IS_A_URL  d=love  d=that  d=grace,  d=struggling  d=sinner/saint,  d=lover  d=of  d=all  d=peoples,  d=follower  d=of  d=a  d=lamb,  d=teller  d=of  d=hope,  d=believer  d=of  d=miracles,  d=provider  d=of  d=rest  d=to  d=the  d=ugly  d=birds! 

In [39]:
# How big is vocabulary if we keep punctuation and urls?
tokens_list = [tweet2tokens(t, use_descr=True, lowercase=True,
                            keep_punctuation=True, descr_prefix='d=',
                            collapse_urls=False, collapse_mentions=True)
              for t in tweets]

vocabulary = make_vocabulary(tokens_list)

29353 unique terms in vocabulary


In [40]:
# How big is vocabulary if we keep punctuation and urls and mentions?
tokens_list = [tweet2tokens(t, use_descr=True, lowercase=True,
                            keep_punctuation=True, descr_prefix='d=',
                            collapse_urls=False, collapse_mentions=False)
              for t in tweets]

vocabulary = make_vocabulary(tokens_list)

32801 unique terms in vocabulary


## Feature Vector Matrix

Create a matrix $X$ where $X[i,j]$ is the frequency of term $j$ in tweet $i$.

$$
X = \begin{pmatrix}
~ & \hbox{term}_1 & \hbox{term}_2 & \hbox{term}_3 & \hbox{term}_4 \\
\hbox{tweet}_1 & 1  &  0  &  0 & 0 \\
\hbox{tweet}_2 & 0  &  0  &  0 & 2 \\
\hbox{tweet}_3 & 1  &  1  &  0 & 0 \\
\end{pmatrix}
$$



## Sparse Matrices

$$
X = \begin{pmatrix}
~ & \hbox{term}_1 & \hbox{term}_2 & \hbox{term}_3 & \hbox{term}_4 \\
\hbox{tweet}_1 & 1  &  0  &  0 & 0 \\
\hbox{tweet}_2 & 0  &  0  &  0 & 2 \\
\hbox{tweet}_3 & 1  &  1  &  0 & 0 \\
\end{pmatrix}
$$

$X$ is mostly $0$ for text problems.

## List of List (LIL) Matrix

Store a linked list of (index, value) pairs for each row.

$$
X = \begin{pmatrix}
\hbox{tweet}_1 & (0, 1)\\
\hbox{tweet}_2 & (3,2)\\
\hbox{tweet}_3 & (0,1), (1,1)\\
\end{pmatrix}
$$

**Advantage:** Fast to construct: append to list in constant time.

**Disadvantage:** Slow random access for matrix-vector product.

E.g., $\hat{z} = X\cdot \hat{\beta}$ to classify tweets using a learned weight vector $\beta$

$\hat{z}[i] = \sum_j X[i,j] * \beta[j]$

## Compressed Sparse Row (CSR) Matrix


$$
X = \begin{pmatrix}
~ & \hbox{term}_1 & \hbox{term}_2 & \hbox{term}_3 & \hbox{term}_4 \\
\hbox{tweet}_1 & 1  &  0  &  0 & 0 \\
\hbox{tweet}_2 & 0  &  0  &  0 & 2 \\
\hbox{tweet}_3 & 1  &  1  &  0 & 0 \\
\hbox{tweet}_4 & 1  &  0  &  0 & 0 \\
\end{pmatrix}
$$

CSR Matrix is an object with three attributes: 
- **val:** $\{1,2,1,1,1\}$  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; *list of all non-zero values*  
- **col_ind:** $\{0,3,0,1,0\}$ &nbsp; *column index for each non-zero value* (e.g., first non-zero value (1) is in column 0) 
- **row_ptr:** $\{0,1,2,4\}$ &nbsp;&nbsp;&nbsp; *index into **col_ind** where each row starts* (e.g., tweet3, term1 corresponds to col_ind[2])

Allows efficient row access (good for us, since each row is a tweet)

In [41]:
# Convert features to a sparse matrix X.
# X[i,j] is the frequency of term j in tweet i
# 
from scipy.sparse import lil_matrix

def make_feature_matrix(tokens_list, vocabulary):
    X = lil_matrix((len(tweets), len(vocabulary)))
    for i, tokens in enumerate(tokens_list):
        for token in tokens:
            j = vocabulary[token]
            X[i,j] += 1
    return X.tocsr()  # convert to CSR for more efficient random access.

In [42]:
X = make_feature_matrix(tokens_list, vocabulary)
print('shape of X:', X.shape)

shape of X: (5000, 32801)


In [35]:
help(X)

Help on csr_matrix in module scipy.sparse.csr object:

class csr_matrix(scipy.sparse.compressed._cs_matrix, scipy.sparse.sputils.IndexMixin)
 |  Compressed Sparse Row matrix
 |  
 |  This can be instantiated in several ways:
 |      csr_matrix(D)
 |          with a dense matrix or rank-2 ndarray D
 |  
 |      csr_matrix(S)
 |          with another sparse matrix S (equivalent to S.tocsr())
 |  
 |      csr_matrix((M, N), [dtype])
 |          to construct an empty matrix with shape (M, N)
 |          dtype is optional, defaulting to dtype='d'.
 |  
 |      csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
 |          where ``data``, ``row_ind`` and ``col_ind`` satisfy the
 |          relationship ``a[row_ind[k], col_ind[k]] = data[k]``.
 |  
 |      csr_matrix((data, indices, indptr), [shape=(M, N)])
 |          is the standard CSR representation where the column indices for
 |          row i are stored in ``indices[indptr[i]:indptr[i+1]]`` and their
 |          corresponding value

In [43]:
# How is tweet 201 stored?
X[201]

<1x32801 sparse matrix of type '<class 'numpy.float64'>'
	with 35 stored elements in Compressed Sparse Row format>

In [44]:
help(X[201].nonzero)

Help on method nonzero in module scipy.sparse.base:

nonzero() method of scipy.sparse.csr.csr_matrix instance
    nonzero indices
    
    Returns a tuple of arrays (row,col) containing the indices
    of the non-zero elements of the matrix.
    
    Examples
    --------
    >>> from scipy.sparse import csr_matrix
    >>> A = csr_matrix([[1,2,0],[0,0,3],[4,0,5]])
    >>> A.nonzero()
    (array([0, 0, 1, 2, 2]), array([0, 1, 2, 0, 2]))



In [46]:
# non-zero indices of terms used in tweet 200.
X[201].nonzero()[1]

array([  31,   80,  147,  150,  164,  436,  682,  690,  726,  960, 1156,
       1173, 2511, 2516, 2696, 2697, 2698, 2699, 2700, 2701, 2702, 2703,
       2704, 2705, 2706, 2707, 2708, 2709, 2710, 2711, 2712, 2713, 2714,
       2715, 2716], dtype=int32)

In [47]:
# term counts for tweet 200.
X[201].data

array([ 1.,  5.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [51]:
# What word does each term index correspond to?
# Convert term->index dict into index->term dict
index2term = {i: t for t, i in vocabulary.items()}
print(index2term[80])
print(X[201, 80])
# So, the term "d=of" (index 80) appears in user 201's tweet five times

d=of
5.0


In [53]:
# d=the appears one time.
print(index2term[31])
print(X[201, 31])

d=the
1.0


How do CSR matrices access row values?

Recall:

CSR Matrix is an object with three attributes: 
- **val:** $\{1,2,1,1\}$  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; *list of all non-zero values*  
- **col_ind:** $\{0,3,0,1\}$ &nbsp; *column index for each non-zero value* (e.g., first non-zero value (1) is in column 0) 
- **row_ptr (ind_ptr):** $\{0,1,2\}$ &nbsp;&nbsp;&nbsp; *index into **col_ind** where each row starts* (e.g., tweet3, term1 corresponds to col_ind[2])

In [54]:
# Recall: numpy array slices.
import numpy as np
a = np.array([0, 100, 200, 300, 400, 500])
a[2:5]  # get elements at positions 2,3

array([200, 300, 400])

In [59]:
#where the column indices for row i are stored in indices[indptr[i]:indptr[i+1]] 
#and their corresponding values are stored in data[indptr[i]:indptr[i+1]]
#so the X.indptr[r] is the number of elements stored before r'th tweet begins to store
print('tweet 200 starts at col_ind=', X.indptr[201])
print('tweet 201 starts at col_ind=', X.indptr[202])
print('so, the columns that are non-zero for tweet 201 are:')
print(X.indices[X.indptr[201]:X.indptr[202]])
print('and the data associated with those cells are:')
print(X.data[X.indptr[201]:X.indptr[202]])

tweet 200 starts at col_ind= 4104
tweet 201 starts at col_ind= 4139
so, the columns that are non-zero for tweet 201 are:
[  31   80  147  150  164  436  682  690  726  960 1156 1173 2511 2516 2696
 2697 2698 2699 2700 2701 2702 2703 2704 2705 2706 2707 2708 2709 2710 2711
 2712 2713 2714 2715 2716]
and the data associated with those cells are:
[ 1.  5.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


In [56]:
print('tweet 0:\n', X[0], '\n')
print('tweet 1:\n', X[1], '\n')
print('tweet 2:\n', X[2])

tweet 0:
   (0, 0)	1.0
  (0, 1)	1.0
  (0, 2)	1.0
  (0, 3)	1.0
  (0, 4)	1.0
  (0, 5)	1.0
  (0, 6)	1.0
  (0, 7)	1.0
  (0, 8)	1.0
  (0, 9)	1.0
  (0, 10)	1.0
  (0, 11)	1.0
  (0, 12)	1.0
  (0, 13)	1.0
  (0, 14)	1.0
  (0, 15)	1.0
  (0, 16)	1.0
  (0, 17)	1.0
  (0, 18)	1.0
  (0, 19)	1.0
  (0, 20)	1.0
  (0, 21)	1.0
  (0, 22)	1.0
  (0, 23)	1.0
  (0, 24)	1.0
  (0, 25)	1.0
  (0, 26)	1.0
  (0, 27)	1.0
  (0, 28)	1.0
  (0, 29)	1.0
  (0, 30)	1.0
  (0, 31)	1.0
  (0, 32)	1.0
  (0, 33)	1.0
  (0, 34)	1.0
  (0, 35)	1.0 

tweet 1:
   (0, 36)	1.0
  (0, 37)	1.0
  (0, 38)	1.0
  (0, 39)	1.0
  (0, 40)	2.0
  (0, 41)	1.0
  (0, 42)	1.0
  (0, 43)	2.0
  (0, 44)	1.0
  (0, 45)	1.0
  (0, 46)	1.0
  (0, 47)	1.0
  (0, 48)	1.0
  (0, 49)	1.0
  (0, 50)	1.0
  (0, 51)	1.0
  (0, 52)	1.0
  (0, 53)	1.0
  (0, 54)	1.0
  (0, 55)	1.0
  (0, 56)	1.0
  (0, 57)	1.0
  (0, 58)	1.0
  (0, 59)	1.0
  (0, 60)	1.0
  (0, 61)	1.0 

tweet 2:
   (0, 8)	1.0
  (0, 9)	1.0
  (0, 62)	1.0
  (0, 63)	1.0
  (0, 64)	1.0
  (0, 65)	1.0
  (0, 66)	1.0
  (0, 67)	1.

**Efficient matrix vector product:**

In [58]:
# Compute z = X * \beta, where X is a CSR matrix.
import numpy as np
beta = np.ones(len(vocabulary))  # assume Beta = vector of 1's
z = np.zeros(len(tweets))
for i in range(len(tweets)):  # for each row.
    for j in range(X.indptr[i], X.indptr[i+1]): # for each col.
        colidx = X.indices[j]
        z[i] += beta[colidx] * X.data[j]
print('X * beta for tweet 201=', z[201])
print('which is the same as the sum %.1f, since beta=[1...1]' % X[201].sum())

X * beta for tweet 201= 39.0
which is the same as the sum 39.0, since beta=[1...1]


**4.) Create a list of gender labels.**

In [60]:
# y is a 1d numpy array of gender labels.
# Let 1=Female, 0=Male.
import numpy as np

def get_gender(tweet, male_names, female_names):
    name = get_first_name(tweet)
    if name in female_names:
        return 1
    elif name in male_names:
        return 0
    else:
        return -1
    
y = np.array([get_gender(t, male_names, female_names) for t in tweets])
print('gender labels:', Counter(y))

gender labels: Counter({0: 2582, 1: 2418})


**5.) Fit a Logistic Regression classifier to predict gender from profile/tweet.**

In [61]:
# Do 5-fold cross-validation
# http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.KFold.html
from sklearn.cross_validation import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

def do_cross_val(X, y, nfolds):
    """ Compute average cross-validation acccuracy."""
    cv = KFold(len(y), nfolds)
    accuracies = []
    for train_idx, test_idx in cv:
        clf = LogisticRegression()
        clf.fit(X[train_idx], y[train_idx])
        predicted = clf.predict(X[test_idx])
        acc = accuracy_score(y[test_idx], predicted)
        accuracies.append(acc)
    avg = np.mean(accuracies)
    return avg

/Users/Dan/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [62]:
print('avg accuracy', do_cross_val(X, y, 5))

avg accuracy 0.6842


In [63]:
# Fitting model with CSR much, much faster than with LIL.
from timeit import timeit
print('CSR TIME')
timeit("do_cross_val(X.tocsr(), y, 2)", number=5,
       setup="from __main__ import do_cross_val, X, y")

CSR TIME


0.6158271009917371

In [64]:
print('LIL TIME')
timeit("do_cross_val(X.tolil(), y, 2)", number=5,
       setup="from __main__ import do_cross_val, X, y")

LIL TIME


252.9111922950251

In [65]:
# How does tokenization affect accuracy?
# Collapse urls and mentions; ignore description prefix.
def run_all(tweets, use_descr=True, lowercase=True,
            keep_punctuation=True, descr_prefix=None,
            collapse_urls=True, collapse_mentions=True):
    
    tokens_list = [tweet2tokens(t, use_descr, lowercase,
                            keep_punctuation, descr_prefix,
                            collapse_urls, collapse_mentions)
                  for t in tweets]
    vocabulary = make_vocabulary(tokens_list)
    X = make_feature_matrix(tokens_list, vocabulary)
    acc = do_cross_val(X, y, 5)
    print('acc=', acc)
    return acc

In [66]:
argnames = ['use_descr', 'lower', 'punct', 'prefix', 'url', 'mention']
option_iter = product(use_descr_opts, lowercase_opts,
                       keep_punctuation_opts,
                       descr_prefix_opts, url_opts,
                       mention_opts)
results = []
for options in option_iter:
    option_str = '\t'.join('%s=%s' % (name, opt) for name, opt in zip(argnames, options))
    print(option_str)
    acc = run_all(tweets, *options)
    results.append((acc, options))
    print

use_descr=True	lower=True	punct=True	prefix=d=	url=True	mention=True
27237 unique terms in vocabulary
acc= 0.6826
use_descr=True	lower=True	punct=True	prefix=d=	url=True	mention=False
30685 unique terms in vocabulary
acc= 0.6848
use_descr=True	lower=True	punct=True	prefix=d=	url=False	mention=True
29353 unique terms in vocabulary
acc= 0.6832
use_descr=True	lower=True	punct=True	prefix=d=	url=False	mention=False
32801 unique terms in vocabulary
acc= 0.6842
use_descr=True	lower=True	punct=True	prefix=	url=True	mention=True
24001 unique terms in vocabulary
acc= 0.6834
use_descr=True	lower=True	punct=True	prefix=	url=True	mention=False
27416 unique terms in vocabulary
acc= 0.6906
use_descr=True	lower=True	punct=True	prefix=	url=False	mention=True
26118 unique terms in vocabulary
acc= 0.6866
use_descr=True	lower=True	punct=True	prefix=	url=False	mention=False
29533 unique terms in vocabulary
acc= 0.6888
use_descr=True	lower=True	punct=False	prefix=d=	url=True	mention=True
19261 unique terms

In [67]:
for r in sorted(results, reverse=True):
    print('%.4f' % r[0], '  '.join('%s=%s' % (name, opt) for name, opt in zip(argnames, r[1])))

0.6918 use_descr=True  lower=False  punct=False  prefix=  url=True  mention=True
0.6918 use_descr=True  lower=False  punct=False  prefix=  url=False  mention=False
0.6914 use_descr=True  lower=False  punct=False  prefix=d=  url=True  mention=True
0.6910 use_descr=True  lower=False  punct=False  prefix=  url=True  mention=False
0.6906 use_descr=True  lower=True  punct=True  prefix=  url=True  mention=False
0.6898 use_descr=True  lower=False  punct=True  prefix=  url=False  mention=False
0.6898 use_descr=True  lower=True  punct=False  prefix=  url=False  mention=False
0.6896 use_descr=True  lower=False  punct=False  prefix=d=  url=False  mention=True
0.6896 use_descr=True  lower=False  punct=False  prefix=  url=False  mention=True
0.6892 use_descr=True  lower=True  punct=False  prefix=  url=False  mention=True
0.6892 use_descr=True  lower=True  punct=False  prefix=d=  url=False  mention=False
0.6892 use_descr=True  lower=False  punct=True  prefix=  url=True  mention=True
0.6890 use_descr

## Error Analysis

- Which ones do we get wrong?
- Are there obvious reasons?